In [ ]:
from FlightRadar24.api import FlightRadar24API

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import SparkSession, Row

In [ ]:
fr_api = FlightRadar24API()

spark = (SparkSession
            .builder
            .getOrCreate())


In [ ]:
def extract_zones_toDF():
    zones = fr_api.get_zones()
    df_zones = spark.createDataFrame(zones)
    return df_zones

In [190]:
#df_zones = extract_zones_toDF()

schema_zones = StructType([
    StructField("Continent", StringType(), True),
    StructField("br_x", StringType(), True),
    StructField("br_y", StringType(), True),
    StructField("tl_x", StringType(), True),
    StructField("tl_y", FloatType(), True),
    StructField("Subzone1", FloatType(), True),
    StructField("br_x_1", StringType(), True),
    StructField("br_y_1", StringType(), True),
    StructField("tl_x_1", StringType(), True),
    StructField("tl_y_1", FloatType(), True),
    StructField("Subzone2", FloatType(), True),
    StructField("br_x_2", StringType(), True),
    StructField("br_y_2", StringType(), True),
    StructField("tl_x_2", StringType(), True),
    StructField("tl_y_2", FloatType(), True),
    ])

zones = fr_api.get_zones()
#df = spark.read.json('/test.json')
#df = spark.read.option("multiline", "true").json('/test.json')
#df.printSchema()


type(zones)

dict

In [219]:
schema = StructType([
  StructField('Continent', StringType(), True)
  #StructField('Continent_tl_y', StringType(), True),
  #StructField('Continent_tl_x', StringType(), True),
  #StructField('Continent_br_y', StringType(), True),
  #StructField('Continent_br_x', StringType(), True)
  ])



continents = ['europe',
 'northamerica',
 'southamerica',
 'oceania',
 'asia',
 'africa',
 'atlantic',
 'maldives',
 'northatlantic']

df = spark.createDataFrame(continents, schema=schema)#.toDF(columns:_*)
df.show()
"""for x in zones:
    continents.append(x)"""
"""if 'subzones' in zones[x]:
        print(zones[x]['subzones'])
    df.withColumn("bonus_amount", df.salary*0.3).show()"""
#continents


TypeError: StructType can not accept object 'europe' in type <class 'str'>

In [ ]:
def extract_airlines_toDF():
    airlines = fr_api.get_airlines()
    df_airlines = spark.createDataFrame(airlines)
    return df_airlines

La colonne 'alt' contient qq données comme "-1" qui sont catégorisées comme string et pas int.
La colonne 'lon' et 'lan contient qq données catégorisées comme int et pas float comme la majorité.

In [ ]:
def convert_airports(lst):
    result = [dict( [a, int(x)] if a == "alt" else [a, float(x)] if a == 'lon' or a == 'lat' else [a, x] for a, x in b.items()) for b in lst]
    return result

In [ ]:
def extract_airports_toDF():
    airports = fr_api.get_airports()
    airports_typed_columns = convert_airports(airports)

    schema_airports = StructType([
    StructField("name", StringType(), True),
    StructField("iata", StringType(), True),
    StructField("icao", StringType(), True),
    StructField("country", StringType(), True),
    StructField("lat", FloatType(), True),
    StructField("lon", FloatType(), True),
    StructField("alt", IntegerType(), True)
    ])
    df_airports = spark.createDataFrame(airports_typed_columns, schema=schema_airports)

    return df_airports

In [ ]:
def convert_flights(lst):
    result = [b.__dict__ for b in lst]
    return result

In [ ]:
def extract_flights_toDF():
   
   flights = fr_api.get_flights()
   flights = convert_flights(flights)
   
   schema = StructType([
   StructField("id", StringType(), True),
   StructField("icao_24bit", StringType(), True),
   StructField("latitude", FloatType(), True),
   StructField("longitude", FloatType(), True),
   StructField("heading", IntegerType(), True),
   StructField("altitude", IntegerType(), True),
   StructField("ground_speed", IntegerType(), True),
   StructField("squawk", StringType(), True),
   StructField("aircraft_code", StringType(), True),
   StructField("registration", StringType(), True),
   StructField("time", IntegerType(), True),
   StructField("origin_airport_iata", StringType(), True),
   StructField("destination_airport_iata", StringType(), True),
   StructField("number", StringType(), True),
   StructField("airline_iata", StringType(), True),
   StructField("on_ground", IntegerType(), True),
   StructField("vertical_speed", IntegerType(), True),
   StructField("callsign", StringType(), True),
   StructField("airline_icao", StringType(), True)
   ])

   flights = fr_api.get_flights()
   df_flights = spark.createDataFrame(flights, schema=schema)
   return df_flights

In [ ]:
df_flights = extract_flights_toDF()
df_airports = extract_airports_toDF()
df_airlines = extract_airlines_toDF()
#df_zones = extract_zones_toDF()

#df_flights.show(10)
df_airports.show(10, False)

Chercher pour anomalies sur df_airlines

In [ ]:
def check_column_quality(df, columnName ):
    vals = df.select(columnName).count()
    distinct_vals = df.select(columnName).distinct().count()
    null_vals = df.filter((df[columnName] == "") | (df[columnName] == "N/A") | (df[columnName] == None) ).count()
    print(f"Column {(columnName)} values : {(vals)}")
    print(f"Column {(columnName)} distinct values : {(distinct_vals)}")
    print(f"Column {(columnName)} null values : {(null_vals)}\n")

In [ ]:
def check_df_quality(df, df_name):
    vals = df.count() 
    distinct_vals = df.distinct().count() 
    print(f"Dataframe {(df_name)}  values : {(vals)}")
    print(f"Dataframe {(df_name)} distinct values : {(distinct_vals)}\n")
    
    for col in df.dtypes:
        check_column_quality(df, col[0])
    print("------------------------------------------------")

In [ ]:
check_df_quality(df_airlines, "Airlines")

In [ ]:
check_df_quality(df_airports, "Airports")

In [ ]:
check_df_quality(df_flights, "Flights")

Autres operations de filtrage sur le df_airlines pour afficher  des doublons sur des groupes de colonnes

In [ ]:
df_airlines \
    .groupby(['Name', 'Code']) \
    .count() \
    .sort('count', ascending=False) \
    #.show(5,False)


df_airlines \
    .select(['ICAO', 'Code', 'Name',]) \
    .where("Name == 'Babcock Scandinavian AirAmbulance'") \
    #.show(10,False)

### Tache nr 1: La compagnie avec le + de vols en cours

In [ ]:
df_flights = extract_flights_toDF()
df_flights = df_flights.filter((df_flights.airline_icao != 'N/A') & (df_flights.on_ground == 0))

airline = df_flights.groupby(['airline_icao']) \
    .count() \
    .sort('count', ascending=False).first()

df_airlines.filter(df_airlines.ICAO == airline['airline_icao']).show()


### Tache nr 2: Pour chaque continent, la compagnie avec le + de vols régionaux actifs (continent d'origine == continent de destination)

In [ ]:
df_flights = extract_flights_toDF()
df_airports = extract_airports_toDF()
df_airlines = extract_airlines_toDF()
df_zones = extract_zones_toDF()
df_zones.show(5,False)